In [2]:
!ls -al /kaggle/input/

In [3]:
!ls -al /kaggle/input/landmark-recognition-2020/

In [4]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [5]:
df = pd.read_csv('/kaggle/input/landmark-recognition-2020/train.csv')
df.head()

In [6]:
# 전체 데이터를 랜덤하게 섞고 새로운 인덱스 부여
df = df.sample(frac=1).reset_index(drop=True)
df.head()

In [7]:
print(len(df))
# 파일갯수가 너무 많아서 캐근 노트북 시간이 오래걸려
# 데이터를 조금만 사용
df = df[df['landmark_id'].map(df['landmark_id'].value_counts())==99]
print(len(df))
print(len(df.landmark_id.unique()))

In [8]:
plt.hist(df.landmark_id, bins=1000)
plt.show()

In [9]:
# landmark ID 고유의 아이디number
# 이것을 카테고리화 시켜줘야 할듯 하다.
# (==50)일때 카테고리 갯수 는 총 169개 -> 들쑥날쑥 값 많았는데
# (==99)일때는 안바꿔줘도 일정하네...
df.landmark_id = df.landmark_id.astype('category')
df['class_id'] = df.landmark_id.cat.codes
# shuffle
df = df.sample(frac=1).reset_index(drop=True)
df.head()

In [10]:
plt.hist(df.class_id, bins=1000)
plt.show()
# 확인!

In [11]:
# 넘파이 배열로 바꾼 후 shape와 형태 확인
image_ids = df.id.to_numpy()
print(image_ids.shape)
print(image_ids[:3])

class_ids = df.class_id.to_numpy()
print(class_ids.shape)
print(class_ids[:3])

In [12]:
image_ids[0]

In [13]:
# 첫 시도때 확인절차 1
# image_id = '17660ef415d37059'
# full_path = '/kaggle/input/landmark-recognition-2020/{}/{}/{}/{}/{}.jpg'.format('train',image_id[0],image_id[1],image_id[2],image_id)
# print(full_path)

In [14]:
# 확인절차2
# !ls  -al {full_path}

In [15]:
# 확인절차3
from PIL import Image

image_id = '17660ef415d37059'
full_path = '/kaggle/input/landmark-recognition-2020/{}/{}/{}/{}/{}.jpg'.format('train',image_id[0],image_id[1],image_id[2],image_id)
print(full_path)

img = Image.open(full_path)
image = img.resize((224, 224))
image = np.array(image)

plt.imshow(image)
plt.show()

In [16]:
from PIL import Image

# 이코드들을 함수화
def load_image(image_id):
  full_path = '/kaggle/input/landmark-recognition-2020/{}/{}/{}/{}/{}.jpg'.format(
      'train',image_id[0],image_id[1],image_id[2],image_id)
 
  img = Image.open(full_path)
  image = img.resize((224, 224))
  return np.array(image)

image = load_image('c613cbd9c6b257b3')
plt.imshow(image)
plt.show()

In [17]:
x = []
y = []
print("total {} images.".format(len(image_ids)))
for i, image_id in enumerate(image_ids):
    if i%100==99:
        print("{} images loaded".format(i+1))
    x.append(load_image(image_id))
    y.append(class_ids[i])

In [18]:
# 다시 배열 정리
x = np.array(x)
y = np.array(y)

In [19]:
# 슬라이싱
i = int(len(x)*0.8)

train_x, test_x = x[:i], x[i:]
train_y, test_y = y[:i], y[i:]

In [20]:
# 중간 확인
for i in range(3):
    plt.imshow(train_x[i])
    plt.show()

In [21]:
import os 
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [23]:
import os

import matplotlib.pyplot as plt
import numpy as np

from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

from tensorflow.keras.applications import efficientnet
from tensorflow.keras.applications import EfficientNetB2

preprocessor = efficientnet.preprocess_input

conv_layers = EfficientNetB2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
conv_layers.trainable = False

model = keras.Sequential()

model.add(conv_layers)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(39, activation='softmax')) #<-----아웃풋 카테고리 수에 맞게 조절

model.compile(loss='sparse_categorical_crossentropy', optimizer="RMSprop", metrics=['acc'])
model.summary()

BATCH_SIZE = 64

train_data_generator = ImageDataGenerator(
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      vertical_flip=False,
      preprocessing_function=preprocessor
).flow(train_x, train_y, batch_size=BATCH_SIZE)

test_data_generator = ImageDataGenerator(
      preprocessing_function=preprocessor
).flow(test_x, test_y, batch_size=BATCH_SIZE)


history = model.fit(
      train_data_generator,
      validation_data=test_data_generator,
      epochs=10,
      verbose=1
)


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()


loss, acc = model.evaluate(test_data_generator)
print("loss=", loss)
print("acc=", acc)



test_x, test_y = test_data_generator.next()
y_ = model.predict(test_x)
predicted = np.argmax(y_, axis=-1)

plt.plot(test_y[:100], "o")
plt.plot(predicted[:100], '.')
plt.show()


# custom_labels = list(test_data_generator.class_indices.keys())
# print("label category index =", test_y[0])
# print("predicted category index =", predicted[0])
# print("predicted category name =", custom_labels[predicted[0]])
for i in range(5):
    plt.imshow(test_x[i].astype(np.int))
    plt.show()
    print("label category index =", test_y[i])
    print("predicted category index = ", predicted[i])

In [ ]:
model.save('gomodel.h5')